Clear the dataset obtained in the previous stages in accordance with the steps from the lecture

Perform data analysis (optional)

Choose a method and perform normalization


In [1]:
!pip install rdkit
!pip install mordred

In [61]:
import pandas as pd
from rdkit import Chem
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestRegressor

In [17]:
df = pd.read_csv('qm9_1.csv')

In [14]:
df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_109095,CCC1=CC=CCCC1,3.03486,1.07556,0.86799,0.6913,97.97,-0.2028,-0.0147,0.1880,1437.9297,0.207597,-351.182105,-351.172934,-351.171990,-351.216386,35.192
1,gdb_126739,CCn1ccc(n1)OC,3.12802,1.11330,0.87300,1.9682,79.80,-0.2077,0.0319,0.2396,1364.4040,0.159883,-419.227054,-419.217941,-419.216997,-419.261636,32.830
2,gdb_61921,CC(NCC#N)C1CN1,3.23759,0.78411,0.66376,4.9958,80.82,-0.2486,0.0209,0.2695,1669.6409,0.170769,-399.297998,-399.288094,-399.287150,-399.333993,35.251
3,gdb_17599,CC(=O)C(=O)N1CC1,3.92086,1.63625,1.19988,1.3069,64.00,-0.2403,-0.0671,0.1732,978.7413,0.116682,-399.810511,-399.802101,-399.801156,-399.844284,28.769
4,gdb_94340,OC1CCC11CC1C#C,2.61019,1.28615,1.07855,2.0761,82.73,-0.2360,0.0475,0.2835,1195.5554,0.157575,-385.829753,-385.820728,-385.819784,-385.863532,34.636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,gdb_50348,O=CC1OC2CCC1O2,2.96810,1.58096,1.50542,2.2867,66.90,-0.2542,-0.0346,0.2196,940.7575,0.137406,-458.974668,-458.967525,-458.966581,-459.006630,27.430
19996,gdb_108076,OCC12OC3CC1(O)C23,2.70275,1.70254,1.31010,0.9403,67.04,-0.2260,0.0509,0.2769,987.0831,0.135449,-458.882355,-458.874610,-458.873666,-458.914492,30.611
19997,gdb_29605,Cc1cocc1NC=O,2.96825,1.29025,0.90440,4.2128,73.70,-0.2117,0.0026,0.2142,1225.3559,0.124686,-437.941730,-437.933372,-437.932428,-437.975111,30.422
19998,gdb_86262,CC1CC(C)C1(C)C#N,1.71625,1.62544,1.12788,4.1443,86.51,-0.2923,0.0344,0.3268,1240.4082,0.193300,-367.235662,-367.225728,-367.224784,-367.269779,37.636


In [18]:
df['gap'] = df.lumo - df.homo
df.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_109095,CCC1=CC=CCCC1,3.03486,1.07556,0.86799,0.6913,97.97,-0.2028,-0.0147,0.1881,1437.9297,0.207597,-351.182105,-351.172934,-351.171990,-351.216386,35.192
1,gdb_126739,CCn1ccc(n1)OC,3.12802,1.11330,0.87300,1.9682,79.80,-0.2077,0.0319,0.2396,1364.4040,0.159883,-419.227054,-419.217941,-419.216997,-419.261636,32.830
2,gdb_61921,CC(NCC#N)C1CN1,3.23759,0.78411,0.66376,4.9958,80.82,-0.2486,0.0209,0.2695,1669.6409,0.170769,-399.297998,-399.288094,-399.287150,-399.333993,35.251
3,gdb_17599,CC(=O)C(=O)N1CC1,3.92086,1.63625,1.19988,1.3069,64.00,-0.2403,-0.0671,0.1732,978.7413,0.116682,-399.810511,-399.802101,-399.801156,-399.844284,28.769
4,gdb_94340,OC1CCC11CC1C#C,2.61019,1.28615,1.07855,2.0761,82.73,-0.2360,0.0475,0.2835,1195.5554,0.157575,-385.829753,-385.820728,-385.819784,-385.863532,34.636


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mol_id  20000 non-null  object 
 1   smiles  20000 non-null  object 
 2   A       20000 non-null  float64
 3   B       20000 non-null  float64
 4   C       20000 non-null  float64
 5   mu      20000 non-null  float64
 6   alpha   20000 non-null  float64
 7   homo    20000 non-null  float64
 8   lumo    20000 non-null  float64
 9   gap     20000 non-null  float64
 10  r2      20000 non-null  float64
 11  zpve    20000 non-null  float64
 12  u0      20000 non-null  float64
 13  u298    20000 non-null  float64
 14  h298    20000 non-null  float64
 15  g298    20000 non-null  float64
 16  cv      20000 non-null  float64
dtypes: float64(15), object(2)
memory usage: 2.6+ MB


число пустых значений

In [20]:
df.isna().sum()

mol_id    0
smiles    0
A         0
B         0
C         0
mu        0
alpha     0
homo      0
lumo      0
gap       0
r2        0
zpve      0
u0        0
u298      0
h298      0
g298      0
cv        0
dtype: int64

число дубликатов

In [21]:
df.duplicated().sum()

0

*Посмотрим что из себя предстваляет набор данных*

In [23]:
df.describe()

,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,46.059972,1.415338,1.130603,2.717544,75.231736,-0.239810,0.010978,0.250788,1188.704460,0.148436,-411.696717,-411.688254,-411.687310,-411.730116,31.583756
std,4681.603555,2.138495,1.418974,1.558652,8.164118,0.021989,0.046908,0.047485,278.114543,0.033241,39.836330,39.836106,39.836106,39.836854,4.050281
min,0.000000,0.362610,0.358090,0.000000,9.460000,-0.360400,-0.175000,0.037600,26.156300,0.016601,-621.423548,-621.414990,-621.414045,-621.457319,6.278000
25%,2.546568,1.092678,0.910902,1.596675,70.460000,-0.252300,-0.023800,0.216200,1020.218400,0.125306,-437.902821,-437.894150,-437.893205,-437.935856,28.938000
50%,3.091785,1.370345,1.081030,2.497100,75.520000,-0.241000,0.011700,0.249100,1146.592850,0.148231,-417.902969,-417.894298,-417.893354,-417.936374,31.530000
75%,3.819047,1.655470,1.279155,3.653225,80.572500,-0.228800,0.048800,0.287800,1307.305750,0.171132,-387.053606,-387.043720,-387.042776,-387.087956,34.212000
max,619867.683100,293.541110,191.393970,20.024500,127.460000,-0.117700,0.106500,0.430600,3139.708600,0.273169,-56.525887,-56.523026,-56.522082,-56.544961,46.386000


Посмотрим распределение переменной "А" (исп.Plotly - для графиков)

In [33]:
import plotly.express as px
from plotly.subplots import make_subplots
colors = ['#FF0000', '#000000', '#800080', '#FFFF00']


In [34]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Histogram for A', 'Violin Plot for A'])

# Histogram
hist_fig = px.histogram(df, x="A", nbins = 60,
                 color_discrete_sequence = colors,
                 opacity = 0.7)

fig.add_trace(hist_fig['data'][0], row=1, col=1) # Adding a graph indicating the location

# Violin plot
violin_fig = px.violin(df, y="A", color_discrete_sequence = colors, box = True)
fig.add_trace(violin_fig['data'][0], row=1, col=2) # Adding a graph indicating the locatio

# Setting up the layout
fig.update_layout(showlegend=False, title_text="Histogram и Violin Plot")

# Graph display
fig.show()

# Получаем индексы выбросов для столбца 'A'

In [38]:
def detect_outliers_iqr(data, threshold=1.5):
    """
    Detect outliers using the Interquartile Range (IQR) method.

    Parameters:
        data (DataFrame): Input DataFrame.
        threshold (float): Multiplier for the IQR to determine the outlier cutoff.

    Returns:
        outliers_dict (dict): Dictionary containing outlier indices for each numeric column.
    """
    outliers_dict = {}
    for column in data.select_dtypes(include='float64').columns:
        quartile_1, quartile_3 = np.percentile(data[column], [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (threshold * iqr)
        upper_bound = quartile_3 + (threshold * iqr)
        outliers = np.where((data[column] < lower_bound) | (data[column] > upper_bound))[0]
        outliers_dict[column] = outliers
    return outliers_dict


Создаём пустой список **descript**, который будет содержать значения дескрипторов для каждой молекулы

In [39]:
# Вызываем функцию для обнаружения выбросов
outliers_dict = detect_outliers_iqr(df)


In [40]:
# Получаем индексы выбросов для столбца 'A'
outliers = outliers_dict['A']


In [41]:
# Удаляем строки с выбросами, превышающими значение 293.6
outliers_to_remove = [idx for idx in outliers if df.loc[idx, 'A'] > 293.6]
df_cleaned = df.drop(outliers_to_remove)

In [42]:
print(df_cleaned)

           mol_id             smiles        A        B        C      mu  \
0      gdb_109095      CCC1=CC=CCCC1  3.03486  1.07556  0.86799  0.6913   
1      gdb_126739      CCn1ccc(n1)OC  3.12802  1.11330  0.87300  1.9682   
2       gdb_61921     CC(NCC#N)C1CN1  3.23759  0.78411  0.66376  4.9958   
3       gdb_17599   CC(=O)C(=O)N1CC1  3.92086  1.63625  1.19988  1.3069   
4       gdb_94340     OC1CCC11CC1C#C  2.61019  1.28615  1.07855  2.0761   
...           ...                ...      ...      ...      ...     ...   
19995   gdb_50348     O=CC1OC2CCC1O2  2.96810  1.58096  1.50542  2.2867   
19996  gdb_108076  OCC12OC3CC1(O)C23  2.70275  1.70254  1.31010  0.9403   
19997   gdb_29605       Cc1cocc1NC=O  2.96825  1.29025  0.90440  4.2128   
19998   gdb_86262   CC1CC(C)C1(C)C#N  1.71625  1.62544  1.12788  4.1443   
19999  gdb_133494   CC(=O)NCC(F)(F)F  3.34013  0.89611  0.84418  3.2517   

       alpha    homo    lumo     gap         r2      zpve          u0  \
0      97.97 -0.2028 -0.01

Тепловая матрица

In [48]:
df=df_cleaned

In [50]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Histogram for A', 'Violin Plot for A'])

# Histogram
hist_fig = px.histogram(df, x="A", nbins = 60,
                 color_discrete_sequence = colors,
                 opacity = 0.7)

fig.add_trace(hist_fig['data'][0], row=1, col=1) # Adding a graph indicating the location

# Violin plot
violin_fig = px.violin(df, y="A", color_discrete_sequence = colors, box = True)
fig.add_trace(violin_fig['data'][0], row=1, col=2) # Adding a graph indicating the locatio

# Setting up the layout
fig.update_layout(showlegend=False, title_text="Histogram и Violin Plot")

# Graph display
fig.show()

 Методом межквартильного размаха (IQR) в словарь outliers_dict, содержащий индексы выбросов для каждого числового столбца

In [52]:
def detect_outliers_iqr(data, threshold=1.5):
    """
    Detect outliers using the Interquartile Range (IQR) method.

    Parameters:
        data (DataFrame): Input DataFrame.
        threshold (float): Multiplier for the IQR to determine the outlier cutoff.

    Returns:
        outliers_dict (dict): Dictionary containing outlier indices for each numeric column.
    """
    outliers_dict = {}
    for column in data.select_dtypes(include='float64').columns:
        quartile_1, quartile_3 = data[column].quantile([0.25, 0.75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (threshold * iqr)
        upper_bound = quartile_3 + (threshold * iqr)
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)].index
        outliers_dict[column] = outliers
    return outliers_dict

In [54]:
common_indices = set(df.index).intersection(outliers)     #пересечение индексов выбросов и индексов

In [55]:
df = df.drop(common_indices)   #удалять только индексы, которые действительно существуют в DataFrame

In [56]:
outliers_dict = detect_outliers_iqr(df)
for column, outliers in outliers_dict.items():
    print(f"Indices of outliers in column '{column}':", outliers)
    print(f"Outlier values in column '{column}':", df.loc[outliers, column].values)

#удалить выбросы, используя словарь outliers_dict
for column, outliers in outliers_dict.items():
    df = df.drop(outliers)

Indices of outliers in column 'A': Index([  237,   258,   513,   582,   601,   602,   634,   765,   931,   938,
       ...
       18924, 19097, 19335, 19617, 19774, 19778, 19845, 19923, 19948, 19988],
      dtype='int64', length=148)
Outlier values in column 'A': [5.24755 5.20035 5.27334 5.25168 5.28125 5.27082 5.26775 5.31344 5.22802
 5.29696 5.22568 5.2976  5.30777 5.26422 5.20095 5.20608 5.22569 5.21683
 5.28216 5.28092 5.32263 5.2589  5.20301 5.22529 5.24322 5.30694 5.19968
 5.31129 5.21008 5.24482 5.26004 5.22111 5.20718 5.20896 5.24697 5.27367
 5.22405 5.25295 5.30069 5.22474 5.29178 5.28844 5.29815 5.31568 5.23696
 5.32435 5.32437 5.31438 5.32841 5.19858 5.32496 5.20665 5.22947 5.2793
 5.2804  5.25301 5.31921 5.31522 5.28565 5.29047 5.20583 5.31544 5.21106
 5.19783 5.20971 5.2078  5.27254 5.2596  5.21252 5.2905  5.28675 5.26225
 5.32052 5.2275  5.31246 5.31964 5.23173 5.24232 5.24787 5.31454 5.3288
 5.32206 5.24061 5.3118  5.30029 5.26886 5.27957 5.32828 5.26813 5.30874
 5.19956

KeyError: '[2164, 4283, 4448, 4621, 6413, 9376, 10814, 11304, 12709, 14825, 14898, 15707, 16162, 16412] not found in axis'

In [57]:
df.reset_index(drop=True, inplace=True)

In [58]:
df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_109095,CCC1=CC=CCCC1,3.03486,1.07556,0.86799,0.6913,97.97,-0.2028,-0.0147,0.1881,1437.9297,0.207597,-351.182105,-351.172934,-351.171990,-351.216386,35.192
1,gdb_126739,CCn1ccc(n1)OC,3.12802,1.11330,0.87300,1.9682,79.80,-0.2077,0.0319,0.2396,1364.4040,0.159883,-419.227054,-419.217941,-419.216997,-419.261636,32.830
2,gdb_61921,CC(NCC#N)C1CN1,3.23759,0.78411,0.66376,4.9958,80.82,-0.2486,0.0209,0.2695,1669.6409,0.170769,-399.297998,-399.288094,-399.287150,-399.333993,35.251
3,gdb_17599,CC(=O)C(=O)N1CC1,3.92086,1.63625,1.19988,1.3069,64.00,-0.2403,-0.0671,0.1732,978.7413,0.116682,-399.810511,-399.802101,-399.801156,-399.844284,28.769
4,gdb_94340,OC1CCC11CC1C#C,2.61019,1.28615,1.07855,2.0761,82.73,-0.2360,0.0475,0.2835,1195.5554,0.157575,-385.829753,-385.820728,-385.819784,-385.863532,34.636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18092,gdb_50348,O=CC1OC2CCC1O2,2.96810,1.58096,1.50542,2.2867,66.90,-0.2542,-0.0346,0.2196,940.7575,0.137406,-458.974668,-458.967525,-458.966581,-459.006630,27.430
18093,gdb_108076,OCC12OC3CC1(O)C23,2.70275,1.70254,1.31010,0.9403,67.04,-0.2260,0.0509,0.2769,987.0831,0.135449,-458.882355,-458.874610,-458.873666,-458.914492,30.611
18094,gdb_29605,Cc1cocc1NC=O,2.96825,1.29025,0.90440,4.2128,73.70,-0.2117,0.0026,0.2143,1225.3559,0.124686,-437.941730,-437.933372,-437.932428,-437.975111,30.422
18095,gdb_86262,CC1CC(C)C1(C)C#N,1.71625,1.62544,1.12788,4.1443,86.51,-0.2923,0.0344,0.3267,1240.4082,0.193300,-367.235662,-367.225728,-367.224784,-367.269779,37.636


приведем данные к диапазону от 0 до 1

In [63]:
scaler = MinMaxScaler()  # Создаем объект MinMaxScaler
normalized_data = scaler.fit_transform(df.iloc[:, 2:])

In [64]:
normalized_data

array([[0.44928198, 0.33014101, 0.24367825, ..., 0.77460079, 0.77459197,
        0.56630894],
       [0.47304007, 0.3486382 , 0.24618098, ..., 0.57956188, 0.57955377,
        0.47479757],
       [0.50098312, 0.18729507, 0.14165609, ..., 0.63668724, 0.6366724 ,
        0.56859479],
       ...,
       [0.43229479, 0.43536521, 0.2618667 , ..., 0.52591742, 0.52591531,
        0.38150401],
       [0.11300396, 0.59964907, 0.37350511, ..., 0.72858831, 0.7285781 ,
        0.66099725],
       [0.52713334, 0.24218869, 0.23178408, ..., 0.1029891 , 0.10297679,
        0.4519391 ]])

In [65]:
# Создаем новый DataFrame с нормализованными данными, исключая первые два столбца
normalized_df = pd.DataFrame(normalized_data, columns=df.columns[2:])

# Объединяем первые два столбца из исходного DataFrame с нормализованными данными
normalized_df = pd.merge(df[df.columns[:2]], normalized_df, left_index=True, right_index=True)


In [67]:
df=normalized_df

In [68]:
df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_109095,CCC1=CC=CCCC1,0.449282,0.330141,0.243678,0.037569,0.764274,0.624945,0.569449,0.382952,0.370827,0.724906,0.774598,0.774601,0.774601,0.774592,0.566309
1,gdb_126739,CCn1ccc(n1)OC,0.473040,0.348638,0.246181,0.106963,0.525791,0.603349,0.734991,0.513995,0.336688,0.524731,0.579559,0.579562,0.579562,0.579554,0.474798
2,gdb_61921,CC(NCC#N)C1CN1,0.500983,0.187295,0.141656,0.271499,0.539178,0.423094,0.695915,0.590076,0.478414,0.570401,0.636682,0.636687,0.636687,0.636672,0.568595
3,gdb_17599,CC(=O)C(=O)N1CC1,0.675234,0.604947,0.409472,0.071024,0.318414,0.459674,0.383304,0.345038,0.157619,0.343490,0.635213,0.635214,0.635214,0.635210,0.317462
4,gdb_94340,OC1CCC11CC1C#C,0.340981,0.433356,0.348863,0.112827,0.564247,0.478625,0.790409,0.625700,0.258289,0.515049,0.675286,0.675289,0.675289,0.675283,0.544768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18092,gdb_50348,O=CC1OC2CCC1O2,0.432257,0.577848,0.562103,0.124272,0.356477,0.398413,0.498757,0.463104,0.139983,0.430434,0.465630,0.465627,0.465627,0.465633,0.265584
18093,gdb_108076,OCC12OC3CC1(O)C23,0.364586,0.637437,0.464532,0.051101,0.358315,0.522697,0.802487,0.608906,0.161493,0.422223,0.465895,0.465893,0.465893,0.465897,0.388826
18094,gdb_29605,Cc1cocc1NC=O,0.432295,0.435365,0.261867,0.228947,0.445728,0.585721,0.630906,0.449618,0.272126,0.377069,0.525917,0.525917,0.525917,0.525915,0.381504
18095,gdb_86262,CC1CC(C)C1(C)C#N,0.113004,0.599649,0.373505,0.225224,0.613860,0.230498,0.743872,0.735623,0.279115,0.664926,0.728583,0.728588,0.728588,0.728578,0.660997


In [69]:

normalized_df.to_csv('normal_data.csv', index=False)